In [2]:
!rm -rf efdl_hw2
!git clone https://github.com/tem-shett/efdl_hw2.git

Cloning into 'efdl_hw2'...
remote: Enumerating objects: 65, done.
remote: Counting objects: 100% (65/65), done.
remote: Compressing objects: 100% (45/45), done.
remote: Total 65 (delta 37), reused 47 (delta 19), pack-reused 0 (from 0)
Receiving objects: 100% (65/65), 17.52 KiB | 5.84 MiB/s, done.
Resolving deltas: 100% (37/37), done.


In [3]:
!nvidia-smi

Mon Feb 16 01:26:40 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 580.105.08             Driver Version: 580.105.08     CUDA Version: 13.0     |
+-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   40C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [4]:
!torchrun --nproc_per_node=2 efdl_hw2/ddp_cifar100.py

W0216 01:26:46.752000 121 torch/distributed/run.py:774] 
W0216 01:26:46.752000 121 torch/distributed/run.py:774] *****************************************
W0216 01:26:46.752000 121 torch/distributed/run.py:774] Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
W0216 01:26:46.752000 121 torch/distributed/run.py:774] *****************************************
/usr/local/lib/python3.12/dist-packages/torch/distributed/distributed_c10d.py:4807: UserWarning: No device id is provided via `init_process_group` or `barrier `. Using the current device set by the user. 
  warnings.warn(  # warn only once
[rank1]:[W216 01:26:52.813513725 ProcessGroupNCCL.cpp:5023] [PG ID 0 PG GUID 0 Rank 1]  using GPU 1 as device used by this process is currently unknown. This can potentially cause a hang if this rank to GPU mapping is incorrect. You can spec

In [5]:
!torchrun --nproc_per_node=2 efdl_hw2/ddp_cifar100_mine.py

W0216 01:30:03.050000 159 torch/distributed/run.py:774] 
W0216 01:30:03.050000 159 torch/distributed/run.py:774] *****************************************
W0216 01:30:03.050000 159 torch/distributed/run.py:774] Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
W0216 01:30:03.050000 159 torch/distributed/run.py:774] *****************************************
/usr/local/lib/python3.12/dist-packages/torch/distributed/distributed_c10d.py:4807: UserWarning: No device id is provided via `init_process_group` or `barrier `. Using the current device set by the user. 
  warnings.warn(  # warn only once
[rank1]:[W216 01:30:06.007827917 ProcessGroupNCCL.cpp:5023] [PG ID 0 PG GUID 0 Rank 1]  using GPU 1 as device used by this process is currently unknown. This can potentially cause a hang if this rank to GPU mapping is incorrect. You can spec

**Значения памяти, времени и лосса оказались практически идентичными. Теперь побенчмаркаем отдельно SyncBatchNorm стандартную и мою**

In [6]:
!python3 efdl_hw2/benchmark.py


Dim      | Batch    | Time (ms)       | Mem (MB)  
--------------------------------------------------
/usr/local/lib/python3.12/dist-packages/torch/distributed/distributed_c10d.py:4807: UserWarning: No device id is provided via `init_process_group` or `barrier `. Using the current device set by the user. 
  warnings.warn(  # warn only once
/usr/local/lib/python3.12/dist-packages/torch/distributed/distributed_c10d.py:4807: UserWarning: No device id is provided via `init_process_group` or `barrier `. Using the current device set by the user. 
  warnings.warn(  # warn only once
128      | 32       | 1.1974          | 0.08      
128      | 64       | 1.1580          | 0.19      
256      | 32       | 1.1487          | 0.23      
256      | 64       | 1.3058          | 0.38      
512      | 32       | 1.3736          | 0.45      
512      | 64       | 1.2931          | 0.76      
1024     | 32       | 1.2088          | 0.90      
1024     | 64       | 1.1893          | 1.53      


In [7]:
!python3 efdl_hw2/benchmark_mine.py


Dim      | Batch    | Time (ms)       | Mem (MB)  
--------------------------------------------------
/usr/local/lib/python3.12/dist-packages/torch/distributed/distributed_c10d.py:4807: UserWarning: No device id is provided via `init_process_group` or `barrier `. Using the current device set by the user. 
  warnings.warn(  # warn only once
/usr/local/lib/python3.12/dist-packages/torch/distributed/distributed_c10d.py:4807: UserWarning: No device id is provided via `init_process_group` or `barrier `. Using the current device set by the user. 
  warnings.warn(  # warn only once
128      | 32       | 1.1287          | 0.10      
128      | 64       | 1.1872          | 0.23      
256      | 32       | 1.2172          | 0.26      
256      | 64       | 1.2288          | 0.45      
512      | 32       | 1.1447          | 0.52      
512      | 64       | 1.3229          | 0.90      
1024     | 32       | 1.1943          | 1.04      
1024     | 64       | 1.1425          | 1.79      


**Получается моя реализация потребляет больше памяти, но при этом чуток быстрее по времени.** Использовал 2 GPU и соответственно 2 процесса.